# RESEARCH PROJECT B
## Testing two steps of a full experiment

### Setting up modules

In [1]:
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style()

In [2]:
import math
import random
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from agents import Animal, FoodPatch, SleepPatch
from model import SleepAnimals
from experiment import *

In [3]:
current_exp_id = 0
generation_number = 0
abm_models = []
generations = []
results = pd.DataFrame()

### Creating genomes of initial population
The array `genomes` represent the genomes of the first 100 individuals. Each gene in the genome has three possible values:  
`'flex' , 'eat' , 'sleep'`

In [4]:
genomes = genome_alternatives(100)
np.shape(genomes)
selected_genomes = genomes

(100, 24)

### Agent-based model generation and execution (first generation)
In this step, an agent-based model (for fitness evaluation) is created for each indivual and stored in the `abm_models` array.
Considering that each model step represents one minute, we run each model for 7 days.

In [5]:
%%time
for current_exp_id in range(100*generation_number + 0, 100*generation_number + 100):
    i = current_exp_id
    abm_models.append( SleepAnimals(i, genome=selected_genomes[i-100*generation_number],  width=40, height=40) )
    for j in range(60*24*7):
        abm_models[i].step()

Wall time: 1min 11s


In [6]:
# a = model[0].datacollector.get_agent_vars_dataframe()
# final_fitness = a.xs(1,level="AgentID")[]

Example code for accessing properties of the agent.
`experiment_id` is the unique id of each model. (There is only one individual per model)

In [7]:
experiment_id = 20
abm_models[experiment_id].schedule.agents[0].fitness
abm_models[experiment_id].schedule.agents[0].fitness_food
abm_models[experiment_id].schedule.agents[0].fitness_sleep

0.49999999998985495

0.9999999999796918

1.8061387908735792e-14

### Data collection and evaluation of fitness of individuals
From the `abm_models` array, a list `current_generation` is constructed in which each row consists of:  
`experiment_id , fitness of the individual on the last step, genome of the agent`    
This list is then sorted in descending order of fitness values.

In [8]:
current_generation = []
for i in range(100*generation_number + 0, 100*generation_number + 100):
    a = [ abm_models[i].model_id , abm_models[i].schedule.agents[0].fitness , 
         abm_models[i].schedule.agents[0].circadian_rythm ]
    current_generation.append( a )

In [9]:
generation_results = dataframe_generation(generation_number , current_generation)
results = results.append(generation_results)

In [10]:
current_generation.sort(key = lambda x : x[1], reverse=True)
generation_number += 1

The array `generations` will store the captured data of each generation.

In [11]:
generations.append( current_generation )

### Evaluation of individuals and creation of new population (2nd generation)
After the end of the first generation, the 20 fittest individuals are selected and the other 80 are discarded.  
From each one of these selected individuals, 5 new individuals are created. Thus, maintaining the population size.  
Each `gene` has a 5% chance of mutation. In case of mutation, it can take any of the other three available three gene values.

In [12]:
# Variables for the mutation probability and the storage of the genomes for the current
# generation. 
p = 0.05
selected_genomes = np.full( (100 , 24) , 'sleep')

In [13]:
# This loop assigns the gene values to the new individuals, from the most fit individuals
# in the previous generation.
# During each gene assignation, mutation can occurr with a chance of 1 mutation per every
# 20 genes.
for i in range(20):
    for j in range(5):
        for k in range(24):
            a = current_generation[i][2][k]
            a = mutation_gene( a , p )
            selected_genomes[int(100/20)*i + j] [k] = a

### Simulation execution for second generation

In [14]:
# Reading some values
generation_number , current_exp_id

(1, 99)

Running the simulation for the whole new generation.

In [15]:
%%time
for current_exp_id in range(100*generation_number + 0, 100*generation_number + 100):
    i = current_exp_id
    abm_models.append( SleepAnimals(i, genome=selected_genomes[i-100*generation_number],  width=40, height=40) )
    for j in range(60*24*7):
        abm_models[i].step()

Wall time: 37 s


Capturing data.

In [16]:
current_generation = []
for i in range(100*generation_number + 0, 100*generation_number + 100):
    a = [ abm_models[i].model_id , abm_models[i].schedule.agents[0].fitness , abm_models[i].schedule.agents[0].circadian_rythm ]
    current_generation.append( a )

In [17]:
generation_results = dataframe_generation(generation_number , current_generation)
results = results.append(generation_results)

In [18]:
current_generation.sort(key = lambda x : x[1], reverse=True)
generation_number += 1

In [19]:
generations.append( current_generation )

Creating new individuals

In [20]:
selected_genomes = np.full( (100 , 24) , 'sleep')
for i in range(20):
    for j in range(5):
        for k in range(24):
            a = current_generation[i][2][k]
            a = mutation_gene( a , p )
            selected_genomes[int(100/20)*i + j] [k] = a

### Some results

In [33]:
generation_number , current_exp_id

(2, 199)

In [32]:
results.loc['1st'].nlargest(20,'fitness')

,u_id,fitness,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
individual,,,,,,,,,,,,,,,,,,,,,
097,96,0.748750,S,E,F,F,F,E,E,F,...,E,E,S,F,E,E,S,F,S,S
020,19,0.743751,S,S,F,S,E,E,E,E,...,E,S,F,S,S,F,F,E,E,E
063,62,0.743751,S,S,S,S,F,F,E,F,...,S,E,F,F,E,E,F,S,S,S
010,9,0.737510,S,S,S,F,S,S,E,F,...,E,F,E,E,E,F,F,F,F,S
054,53,0.735641,S,F,E,S,E,S,E,E,...,E,E,F,S,E,E,F,F,E,E
087,86,0.735641,S,S,S,S,S,E,E,E,...,F,S,F,F,S,E,S,S,F,F
035,34,0.735018,S,E,F,F,E,E,E,S,...,S,E,S,F,F,S,S,E,F,F
043,42,0.735018,S,E,F,F,S,S,S,F,...,S,E,S,S,S,E,S,E,S,F
036,35,0.733151,S,S,F,S,S,E,F,S,...,F,F,E,E,F,S,S,E,E,F


In [31]:
results.loc['2nd'].nlargest(20,'fitness')

,u_id,fitness,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
individual,,,,,,,,,,,,,,,,,,,,,
005,104,0.749375,S,F,F,F,F,E,E,F,...,E,E,S,F,E,E,S,F,S,S
032,131,0.749375,S,E,F,S,E,E,E,S,...,S,E,S,F,F,S,S,E,F,F
074,173,0.749375,S,E,E,E,S,E,S,E,...,E,F,F,E,S,F,F,F,F,F
049,148,0.748750,S,E,S,F,S,E,S,S,...,E,E,F,E,S,E,E,F,F,F
066,165,0.748750,S,S,E,F,F,S,E,F,...,E,S,F,F,E,F,S,E,S,F
045,144,0.748125,S,S,F,S,S,E,F,S,...,F,F,E,E,F,S,S,E,E,F
053,152,0.748125,S,E,S,S,E,F,S,F,...,E,E,F,E,E,S,S,F,F,E
077,176,0.746875,S,S,S,E,F,E,S,E,...,E,S,E,E,E,S,S,S,E,S
064,163,0.746250,S,E,S,S,E,E,F,F,...,S,S,S,E,F,S,S,F,S,E
